# Práctica 1.2. Representaciones vectoriales y clasificación de textos
## García Rivera Bogdan Kaleb MIA-3

Este proyecto  tiene como finalidad realizar una comparativa de los distintos algoritmos de machine learning con el procesamiento de lenguaje natural. Se hace uso de una base de datos de 50k de reviews de peliculas las cuales cada una de ellas tienen la etiqueta 'positive' y 'negative' respectivamente

### Objetivos
* Construir y comparar diferentes representaciones de texto: Word2Vec preentrenado (Google News), Word2Vec entrenado en un corpus propio, Bolsa de palabras (BoW), N-gramas y TF-IDF
* Entrenar y evaluar al menos 3 modelos de clasificación de texto.
* Comparar el desempeño (accuracy y classification report) entre representaciones vectoriales y modelos.


Importando datos y algunas librerias

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

c:\Users\bugy1\anaconda3\envs\pln\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\bugy1\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1


In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import numpy as np

nltk.download('punkt_tab')
nltk.download('stopwords')

df = pd.read_csv(path + '/IMDB Dataset.csv')
df.head()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\bugy1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bugy1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## 1. Preprocesamiento

1. Preprocesamiento del texto


In [3]:
def preprocessing(text):
    text = text.lower()
    # Quitar etiquetas HTML
    text = re.sub(r'<.*?>', '', text)

    # Quitar URLs y menciones
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)

    # Quitar caracteres especiales
    text = re.sub(r"[^a-zA-Z\s']", ' ', text)

    # Quitar numeros
    text = re.sub(r'\d+', '', text)

    # Quitar dobles espacios
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenización
    tokens = word_tokenize(text)

    # Remover stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Aplicar stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

df['tokens'] = df['review'].apply(preprocessing)
df['clean_text'] = df['tokens'].apply(' '.join)

In [4]:
print(f"Ejemplo de tokens: \n{df['tokens'].iloc[0:3][:6]}")

Ejemplo de tokens: 
0    [one, review, mention, watch, oz, episod, 'll,...
1    [wonder, littl, product, film, techniqu, unass...
2    [thought, wonder, way, spend, time, hot, summe...
Name: tokens, dtype: object


2. Dividir el conjunto en train/test (ej. 70/30)

In [5]:
from sklearn.model_selection import train_test_split

X = df['clean_text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

print(f"""Tamaño del conjunto: 
      X_train: {len(X_train)}
      X_test: {len(X_test)}
      y_train: {len(y_train)}
      y_test: {len(y_test)}
      """)

Tamaño del conjunto: 
      X_train: 35000
      X_test: 15000
      y_train: 35000
      y_test: 15000
      


## 2. Representaciones

1.	Bolsa de Palabras (BoW)


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(
    max_features=7000,      
    min_df=5,               
    max_df=0.8,
    ngram_range=(1, 1)     
)

X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

print(f"   X_train_bow shape: {X_train_bow.shape}")
print(f"   X_test_bow shape: {X_test_bow.shape}")
print(f"   Vocabulario: {len(bow_vectorizer.get_feature_names_out())} palabras")

print("\nPrimeras 10 palabras del vocabulario:")
print(bow_vectorizer.get_feature_names_out()[:10])

   X_train_bow shape: (35000, 7000)
   X_test_bow shape: (15000, 7000)
   Vocabulario: 7000 palabras

Primeras 10 palabras del vocabulario:
['aaron' 'abandon' 'abbott' 'abc' 'abduct' 'abil' 'abl' 'aboard' 'abomin'
 'aborigin']


2.	n-gramas


In [7]:
ngram_vectorizer = CountVectorizer(
    max_features=7000,      
    min_df=5,               
    max_df=0.8,
    ngram_range=(1, 3)     
)

X_train_ngrams = ngram_vectorizer.fit_transform(X_train)
X_test_ngrams = ngram_vectorizer.transform(X_test)


print(f"Train shape: {X_train_ngrams.shape}")
print(f"Total n-gramas: {len(ngram_vectorizer.get_feature_names_out())}")
print("\nEjemplos de n-gramas para tipo de sentimiento:")


ngrams_ejemplos = []
for ngram in ngram_vectorizer.get_feature_names_out():
    if any(phrase in ngram for phrase in ['not', 'very', 'really', 'too', 'so', 'never']):
        ngrams_ejemplos.append(ngram)
    if len(ngrams_ejemplos) >= 10:
        break

print(ngrams_ejemplos)

Train shape: (35000, 7000)
Total n-gramas: 7000

Ejemplos de n-gramas para tipo de sentimiento:
['absolut', 'absolut love', 'absolut noth', 'absorb', 'also', 'also enjoy', 'also featur', 'also get', 'also good', 'also great']


3. TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2)
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


print(f"Train shape: {X_train_tfidf.shape}")
print(f"Test shape: {X_test_tfidf.shape}")


print("\nEjemplo de pesos:")
feature_names = tfidf_vectorizer.get_feature_names_out()
primer_review = X_train_tfidf[0].toarray()[0]
top_indices = primer_review.argsort()[-5:][::-1]  # primeras 5 palabras con mayor peso

for idx in top_indices:
    if primer_review[idx] > 0:
        print(f"  {feature_names[idx]}: {primer_review[idx]:.4f}")

Train shape: (35000, 1000)
Test shape: (15000, 1000)

Ejemplo de pesos:
  steal: 0.3536
  train: 0.3454
  drive: 0.3335
  hit: 0.2947
  guy: 0.2270


4. Word2Vec Google News (preentrenado)

In [9]:
import gensim
import gensim.downloader as api

from gensim.models import Word2Vec, KeyedVectors

wv = api.load('word2vec-google-news-300')


In [10]:
# Promedio de un texto
def get_text_embedding(text, model):
    words = text.split()
    vectors = []
    for word in words:
        if word in model:
            vectors.append(model[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)


X_train_w2v = np.array([get_text_embedding(text, wv) for text in X_train])
X_test_w2v = np.array([get_text_embedding(text, wv) for text in X_test])

print(f"Train shape: {X_train_w2v.shape}")
print(f"Test shape: {X_test_w2v.shape}")
print(f"Dimensionalidad de embeddings: {wv.vector_size}")

# Ejemplo de similitud
if 'good' in wv and 'excellent' in wv:
    similitud = wv.similarity('good', 'excellent')
    print(f"\nSimilitud 'good' - 'excellent': {similitud:.3f}")

Train shape: (35000, 300)
Test shape: (15000, 300)
Dimensionalidad de embeddings: 300

Similitud 'good' - 'excellent': 0.644


5. Word2Vec propio (entrenado con el corpus)

In [11]:
w2v_propio = Word2Vec(
    sentences=df['tokens'],  
    vector_size=100,         
    window=5,                # Ventana de contexto
    min_count=5,             
    workers=4
)

# obtención de embeddings (promedio) 
def get_embedding(tokens, model):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X_train_w2v_propio = np.array([get_embedding(tokens, w2v_propio) for tokens in df['tokens'].iloc[X_train.index]])
X_test_w2v_propio = np.array([get_embedding(tokens, w2v_propio) for tokens in df['tokens'].iloc[X_test.index]])


print(f"Train shape: {X_train_w2v_propio.shape}")
print(f"Test shape: {X_test_w2v_propio.shape}")
print(f"Vocabulario: {len(w2v_propio.wv.key_to_index)} palabras")

# Ejemplo de palabras similares
if 'good' in w2v_propio.wv:
    similares = w2v_propio.wv.most_similar('good', topn=3)
    print(f"\nPalabras similares a 'good': {similares}")

Train shape: (35000, 100)
Test shape: (15000, 100)
Vocabulario: 27088 palabras

Palabras similares a 'good': [('decent', 0.7612664103507996), ('great', 0.7122461199760437), ('bad', 0.7096006870269775)]


## 3. Modelos de Clasificación.

Se realizó la elección de 3 modelos de clasificación: Bósques de decisión, SVC y Regresión logistica


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

representaciones = {
    'BOW': (X_train_bow,X_test_bow),
    'N-gramas': (X_train_ngrams,X_test_ngrams), 
    'TF-IDF': (X_train_tfidf,X_test_tfidf),
    'Word2Vec Propio': (X_train_w2v_propio, X_test_w2v_propio),
    'Word2Vec Google': (X_train_w2v,X_test_w2v)  
}

resultados = {}

for rep_name, (X_train_rep, X_test_rep) in representaciones.items():
    resultados[rep_name] = {}
    
    models_rep = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'KNN': KNeighborsClassifier(),
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=100)
    }
    print(f"\n{rep_name}:\n")
    for model_name, model in models_rep.items():
        # Entrenamiento y predicción
        model.fit(X_train_rep, y_train)
        y_train_pred = model.predict(X_train_rep)
        y_test_pred = model.predict(X_test_rep)
        print(f"{model_name}: Entrenado")
        
        # Métricas
        train_acc = accuracy_score(y_train, y_train_pred)
        test_acc = accuracy_score(y_test, y_test_pred)
        report = classification_report(y_test, y_test_pred, output_dict=True, digits=4)
        
        # Guardar resultados
        resultados[rep_name][model_name] = {
            'train_accuracy': train_acc,
            'test_accuracy': test_acc,
            'classification_report': report,
            'model': model  
        }


BOW:

Random Forest: Entrenado
KNN: Entrenado


c:\Users\bugy1\anaconda3\envs\pln\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: Entrenado

N-gramas:

Random Forest: Entrenado
KNN: Entrenado


c:\Users\bugy1\anaconda3\envs\pln\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: Entrenado

TF-IDF:

Random Forest: Entrenado
KNN: Entrenado
Logistic Regression: Entrenado

Word2Vec Propio:

Random Forest: Entrenado
KNN: Entrenado
Logistic Regression: Entrenado

Word2Vec Google:

Random Forest: Entrenado
KNN: Entrenado
Logistic Regression: Entrenado


## 4. Evaluación 

In [19]:
for rep_name, models_dict in resultados.items():
    print(f"\n--- {rep_name} ---")
    
    for model_name, metrics in models_dict.items():
        report = metrics['classification_report']
        
        print(f"\n{model_name}:")
        print(f"  Accuracy Train: {metrics['train_accuracy']:.4f}")
        print(f"  Accuracy Test:  {metrics['test_accuracy']:.4f}")
        
        print(f"\n  Métricas por clase:")
        print(f"    Clase      Precision  Recall  F1-Score")
        print(f"    -------------------------------------")
        
        for class_name in ['negative', 'positive']:
            if class_name in report:
                print(f"    {class_name:9} {report[class_name]['precision']:8.4f} {report[class_name]['recall']:7.4f} {report[class_name]['f1-score']:8.4f}")


--- BOW ---

Random Forest:
  Accuracy Train: 1.0000
  Accuracy Test:  0.8497

  Métricas por clase:
    Clase      Precision  Recall  F1-Score
    -------------------------------------
    negative    0.8468  0.8494   0.8481
    positive    0.8525  0.8499   0.8512

KNN:
  Accuracy Train: 0.7595
  Accuracy Test:  0.6514

  Métricas por clase:
    Clase      Precision  Recall  F1-Score
    -------------------------------------
    negative    0.6464  0.6501   0.6482
    positive    0.6564  0.6527   0.6545

Logistic Regression:
  Accuracy Train: 0.9589
  Accuracy Test:  0.8701

  Métricas por clase:
    Clase      Precision  Recall  F1-Score
    -------------------------------------
    negative    0.8728  0.8629   0.8678
    positive    0.8676  0.8772   0.8724

--- N-gramas ---

Random Forest:
  Accuracy Train: 1.0000
  Accuracy Test:  0.8496

  Métricas por clase:
    Clase      Precision  Recall  F1-Score
    -------------------------------------
    negative    0.8460  0.8504   0.84

# Resultados

## BOW (Bag of Words)

### Random Forest
- **Accuracy Train**: 1.0000
- **Accuracy Test**: 0.8497

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8468    | 0.8494 | 0.8481   |
| positive  | 0.8525    | 0.8499 | 0.8512   |

### KNN
- **Accuracy Train**: 0.7595
- **Accuracy Test**: 0.6514

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.6464    | 0.6501 | 0.6482   |
| positive  | 0.6564    | 0.6527 | 0.6545   |

### Logistic Regression
- **Accuracy Train**: 0.9589
- **Accuracy Test**: 0.8701

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8728    | 0.8629 | 0.8678   |
| positive  | 0.8676    | 0.8772 | 0.8724   |

## N-gramas

### Random Forest
- **Accuracy Train**: 1.0000
- **Accuracy Test**: 0.8496

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8460    | 0.8504 | 0.8482   |
| positive  | 0.8531    | 0.8489 | 0.8510   |

### KNN
- **Accuracy Train**: 0.7390
- **Accuracy Test**: 0.6257

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.6142    | 0.6517 | 0.6324   |
| positive  | 0.6383    | 0.6002 | 0.6187   |

### Logistic Regression
- **Accuracy Train**: 0.9624
- **Accuracy Test**: 0.8721

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8708    | 0.8703 | 0.8706   |
| positive  | 0.8734    | 0.8739 | 0.8737   |

## TF-IDF

### Random Forest
- **Accuracy Train**: 1.0000
- **Accuracy Test**: 0.8344

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8283    | 0.8386 | 0.8334   |
| positive  | 0.8405    | 0.8303 | 0.8353   |

### KNN
- **Accuracy Train**: 0.8297
- **Accuracy Test**: 0.7372

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.7598    | 0.6845 | 0.7202   |
| positive  | 0.7191    | 0.7886 | 0.7523   |

### Logistic Regression
- **Accuracy Train**: 0.8769
- **Accuracy Test**: 0.8664

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8704    | 0.8572 | 0.8638   |
| positive  | 0.8626    | 0.8753 | 0.8689   |

## Word2Vec Propio

### Random Forest
- **Accuracy Train**: 1.0000
- **Accuracy Test**: 0.8364

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8429    | 0.8222 | 0.8324   |
| positive  | 0.8304    | 0.8503 | 0.8402   |

### KNN
- **Accuracy Train**: 0.8685
- **Accuracy Test**: 0.8039

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.7908    | 0.8199 | 0.8051   |
| positive  | 0.8175    | 0.7882 | 0.8026   |

### Logistic Regression
- **Accuracy Train**: 0.8660
- **Accuracy Test**: 0.8618

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8638    | 0.8551 | 0.8594   |
| positive  | 0.8599    | 0.8684 | 0.8641   |

## Word2Vec Google

### Random Forest
- **Accuracy Train**: 1.0000
- **Accuracy Test**: 0.7717

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.7677    | 0.7714 | 0.7696   |
| positive  | 0.7757    | 0.7720 | 0.7739   |

### KNN
- **Accuracy Train**: 0.8281
- **Accuracy Test**: 0.7315

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.7257    | 0.7339 | 0.7298   |
| positive  | 0.7372    | 0.7291 | 0.7331   |

### Logistic Regression
- **Accuracy Train**: 0.8226
- **Accuracy Test**: 0.8141

| Clase     | Precision | Recall | F1-Score |
|-----------|-----------|--------|----------|
| negative  | 0.8111    | 0.8131 | 0.8121   |
| positive  | 0.8171    | 0.8151 | 0.8161   |

Se observa que la representación más eficaz fue la de N-gramas combinada con regresión logística, alcanzando una precisión de 0.8721 en el conjunto de prueba. En segundo lugar, se posicionó el modelo basado en Bag of Words (BoW) con el mismo algoritmo, aunque con un rendimiento ligeramente inferior.

Asimismo, al comparar el modelo Word2Vec entrenado localmente con el preentrenado de Google, se muestran diferencias significativas. El modelo propio mostró un desempeño superior, obteniendo una precisión de aproximadamente 0.8618 con regresión logística, contra 0.8141 alcanzado por el modelo de Google bajo las mismas condiciones.

Cabe destacar que la regresión logística fue el algoritmo con mejor rendimiento en la mayoría de las representaciones evaluadas. El modelo basado en árboles de decisión no fue considerado en el análisis final debido a indicios claros de sobreajuste. Finalmente, se observa que el grado de generalización varía según el algoritmo utilizado, siendo la regresión logística la que presenta una menor diferencia entre los conjuntos de entrenamiento y prueba, lo que parece mostrar una mayor capacidad de correcta predicción.


## Conclusión

En este proyecto se realizó la comparativa de diversas representaciones y el entrenamiento de cada una de ellas. Como se pudo observar, la representación en N-gramas demostró ser la más efectiva para el objetivo el cual era clasificar sentimientos positivos y negativos. El algoritmo donde alcanzó un mayor accuracy fue con regresión logística (cerca del 87%). Estos resultados se deben principalmente a la captura del contexto de los n-gramas. Los bigramas o trigramas permiten al modelo entender dichas reglas linguisticas los cuales son fundamentales para determinar el sentimiento de una manera precisa. 

Con respecto a los embeddings preentrenados y con el corpus propio, si existen diferencias significativas. El modelo propio alcanzó un accuracy aproximado de 86% mientras que el preentrenado obtuvo un aproximado de 81%. La razón es debido a que, el Word2Vec propio se especializó en este dominio específico. El modelo de Google es muy general y contiene embeddings relacionados a un lenguaje periodístico y de noticias, por lo cual no se alinea completamente a este contexto. 

Entre las limitaciones observadas con respecto a BOW, n-gramas y TF-IDF, en el caso de BOW, se sufre una pérdida completa con respecto al orden y el contexto de las palabras. En general esta representación trata cada palabra como independiente, además de generar matrices muy dispersas y una alta dimensionalidad con los embeddings. En el caso de los n-gramas, estos pueden experimentar combinaciones de palabras muy revueltas, además de que el número de características crece de manera exponencial con el tamaño de n, haciendo que algoritmos como random forest experimenten un overffiting.  Finalmente TF-IDF mostró una pérdida en el rendimiento en comparación con BOW simple. Probablemente para estas clasificaciones la frecuencia de las palabras sea importante, quiere decir que puede que este algoritmo esté eliminando señales importantes para la tarea. 

En un proyecto real se podría hacer uso de una combinación de algunas de estas representaciones. Por ejemplo, la representación de n-gramas con regresión logística podría ser la principal. Posterior a eso, la generación de embeddings se podrían entrenar con el corpus propio (Word2Vec propio) y realizar el entrenamiento con el modelo anteriormente dicho. 